<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/graphtrek_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
# install necessary modules
!pip install yfinance
!pip install plotly==5.2.1
!pip install ta
!pip install mplfinance

In [179]:
import pandas as pd
import yfinance as yf

from datetime import datetime, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import requests
from bs4 import BeautifulSoup
import numpy as np
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt


from ta.trend import MACD
from ta.momentum import StochasticOscillator
from ta.momentum import RSIIndicator


from tabulate import tabulate


In [180]:
# method 1: fractal candlestick pattern

def get_stock_price(ticker, from_date):
  #df = yf.download(ticker.ticker, start='2020-01-01')
  #df = df.rename(columns={"Close": "Close1", "Adj Close": "Close"})
  
  #ticker = yf.Ticker(symbol)

  df = ticker.history(start=from_date, interval="1d")
  #print(df.info())
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  #df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]
  df['MA21'] = df['Close'].ewm(span=21, adjust=False).mean()
  df['MA50'] = df['Close'].rolling(window=50).mean()
  df['MA100'] = df['Close'].rolling(window=100).mean()
  df['MA200'] = df['Close'].rolling(window=200).mean()
  return df

def is_support(df,i):
  cond1 = df['Low'][i] < df['Low'][i-1] 
  cond2 = df['Low'][i] < df['Low'][i+1] 
  cond3 = df['Low'][i+1] < df['Low'][i+2] 
  cond4 = df['Low'][i-1] < df['Low'][i-2]
  return (cond1 and cond2 and cond3 and cond4)

def is_resistance(df,i):
  cond1 = df['High'][i] > df['High'][i-1] 
  cond2 = df['High'][i] > df['High'][i+1] 
  cond3 = df['High'][i+1] > df['High'][i+2] 
  cond4 = df['High'][i-1] > df['High'][i-2]
  return (cond1 and cond2 and cond3 and cond4)

def is_far_from_level(value, levels, df):
    ave =  np.mean(df['High'] - df['Low'])
    return np.sum([abs(value - level) < ave for level in levels]) == 0

def plot_all(levels, df):
    fig, ax = plt.subplots(figsize=(16, 9), dpi=300)
    candlestick_ohlc(ax,df.values,width=0.6, colorup='green', colordown='red', alpha=0.8)
    date_format = mpl_dates.DateFormatter('%d %b %Y')
    ax.xaxis.set_major_formatter(date_format)
    for level in levels:
        plt.hlines(level[1], xmin=df['Date'][level[0]], xmax=max(df['Date']), colors='blue', linestyle='--')
    fig.show()

def findNearestGreaterThan(searchVal, inputData):
    diff = inputData - searchVal
    diff[diff<0] = np.inf
    idx = diff.argmin()
    return inputData[idx]


def findNearestLessThan(searchVal, inputData):
    diff = inputData - searchVal
    diff[diff>0] = -np.inf
    idx = diff.argmax()
    return inputData[idx]

def indicators(chart_df):
  # MACD
  macd = MACD(close=chart_df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
  # stochastics
  stoch = StochasticOscillator(high=chart_df['High'],
                             close=chart_df['Close'],
                             low=chart_df['Low'],
                             window=14, 
                             smooth_window=3)

  rsi = RSIIndicator(close=chart_df['Close'], window=14)
  return macd, stoch, rsi

In [181]:
def calculate_levels(chart_df):
  levels = []
  low = 0
  high = np.round(chart_df['Close'].max(),1)
  for i in range(2,len(chart_df)-2):
    if is_support(chart_df,i):
      low = chart_df['Low'][i]
    if is_far_from_level(low, levels, chart_df):
      levels.append(low)
    elif is_resistance(df,i):
      high = chart_df['High'][i]
    if is_far_from_level(high, levels, chart_df):
      levels.append(high)
  levels = sorted(levels, reverse=True)

  last_day_df = chart_df[-1:]
  close_price = np.round(last_day_df['Close'][0],1)

  min_level = np.round(findNearestLessThan(close_price,levels),1)
  if(min_level > close_price):
    min_level = np.round(close_price * 0.8,1)

  max_level = np.round(findNearestGreaterThan(close_price,levels),1)
  if(max_level < close_price):
    max_level = np.round(close_price * 1.2,1)

  print('close_price',close_price,'min_level:',min_level,'max_level:', max_level)
  return levels, min_level, max_level

In [182]:
def options_chain(ticker):

    #tk = yf.Ticker(symbol)
    # Expiration dates
    exps = ticker.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = ticker.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    #options['expirationDate'] = pd.date(options['expirationDate']) + timedelta(days = 1)
    #options['dte'] = (options['expirationDate'] - datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 
             'ask', 
             'strike', 
             'lastPrice', 
             'volume',
             'change',
             'percentChange',
             'openInterest',
             'impliedVolatility']] = options[[
                                   'bid', 
                                   'ask', 
                                   'strike',
                                   'lastPrice',
                                   'volume',
                                   'change',
                                   'percentChange',
                                   'openInterest',
                                   'impliedVolatility']].apply(pd.to_numeric)
    
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    #options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [183]:
def find_level_optionInterests(ticker,close_price,min_level,max_level):
  options_df = options_chain(ticker)
  #expirationDates = options_df['expirationDate'].unique()
  #print(sorted(expirationDates))

  #PUT_options_df = pd.DataFrame()
  #CALL_options_df = pd.DataFrame()

  #for key, value in options_df.items():
  #  date = key
  #  rsi = float(value.get('RSI'))
  #  rsi_data.append([date,rsi])
  print('PUT OPTIONS', 'CLOSED:',close_price, 'SUPPORT -15%:', np.round(min_level * 0.85,2), 'RESISTANCE +15%:', np.round(max_level * 1.15,2))
  PUT_options_df = options_df.query('CALL == False and strike>' + str(min_level * 0.9) + ' and strike<' + str(max_level * 1.1))
  put_index = PUT_options_df["openInterest"].idxmax()
  PUT_options_df = PUT_options_df.drop(columns = ['contractSize', 'currency','change', 'lastTradeDate', 'lastPrice', 'inTheMoney','contractSymbol', 'CALL']) 
  print(tabulate(PUT_options_df.loc[put_index:put_index], headers = 'keys', tablefmt = 'psql'))

  print('CALL OPTIONS', 'CLOSED:',close_price, 'SUPPORT -15%:', np.round(min_level * 0.85,2), 'RESISTANCE +15%:', np.round(max_level * 1.15,2))
  CALL_options_df = options_df.query('CALL == True and strike>' + str(min_level * 0.85) + ' and strike<' + str(max_level * 1.15))
  call_index = CALL_options_df["openInterest"].idxmax()
  CALL_options_df = CALL_options_df.drop(columns = ['contractSize', 'currency', 'change', 'lastTradeDate', 'lastPrice', 'inTheMoney','contractSymbol','CALL']) 
  print(tabulate(CALL_options_df.loc[call_index:call_index], headers = 'keys', tablefmt = 'psql'))
  return PUT_options_df, CALL_options_df

In [184]:
def draw_chart(ticker, df, days):
  last_day_df = df[-1:]
  last_date = last_day_df['Date'].index[0].date()
  close_price = np.round(last_day_df['Close'][0],1)

  ath = np.round(df['Close'].max(),1)
  discount = np.round(ath - close_price,1)
  discount_percent = np.round((discount / close_price) * 100, 1)
  
  chart_df = df.tail(days)

  macd, soch, rsi = indicators(chart_df)
  levels, min_level, max_level = calculate_levels(chart_df)
  PUT_options_df, CALL_options_df = find_level_optionInterests(ticker,close_price,min_level,max_level)

  last_day_df = chart_df[-1:]
  close_price = np.round(last_day_df['Close'][0],1)

  tradingview_link = '<a href="https://in.tradingview.com/chart/66XmQfYy/?symbol=' + ticker.ticker +'">' + ticker.ticker +'</a>'

  title = '<b>' + tradingview_link + '</b> <b>Date:</b>' + str(last_date) + '<br><b>Close:</b>' + str(close_price) + ' <b>Support:</b>' + str(min_level) + ' <b>Resistance:</b>' + str(max_level) + '<br> <b>ATH:</b>' + str(ath) + '$ <b>Discount:</b>' + str(discount) + ' (' + str(discount_percent) + '%)'

  # add subplot properties when initiliazing fig variable
  fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.5,0.1,0.2,0.2],
                    column_titles=[title])
  # Plot OHLC on 1st subplot (using the codes from before)
  fig.add_trace(go.Candlestick(x=chart_df.index,
                             open=chart_df['Open'],
                             high=chart_df['High'],
                             low=chart_df['Low'],
                             close=chart_df['Close'], 
                             name=ticker.ticker,
                             showlegend=True))
  # add moving average traces
  fig.add_trace(go.Scatter(x=chart_df.index, 
                         y=chart_df['MA21'], 
                         line=dict(color='green', width=2), 
                         name='MA 21'))
  fig.add_trace(go.Scatter(x=chart_df.index, 
                         y=chart_df['MA50'], 
                         line=dict(color='blue', width=2), 
                         name='MA 50'))
  fig.add_trace(go.Scatter(x=chart_df.index, 
                         y=chart_df['MA100'], 
                         line=dict(color='orange', width=2), 
                         name='MA 100'))
  fig.add_trace(go.Scatter(x=chart_df.index, 
                         y=chart_df['MA200'], 
                         line=dict(color='red', width=2), 
                         name='MA 200'))
  ath_percent = 0
  
  for idx, level in  enumerate(levels):
      percent = 0
      if idx == 0:
        ath = level
      current_level = level
      if idx > 0:
        prev_level = levels[idx-1]
        diff = prev_level - current_level
        ath_diff = ath - current_level
        percent = (diff / current_level) * 100
        ath_percent =  (ath_diff / current_level) * 100
        #print(percent)
      fig.add_hline(level,line_dash="dot", opacity=0.5, line_width=1, annotation_position="top right", annotation_text='$' + str(np.round(current_level,1)) + ' (' + str(np.round(percent,1)) + '% discount:' + str(np.round(ath_percent,1))+ '%)' )

  # Plot volume trace on 2nd row 
  colors = ['green' if row['Open'] - row['Close'] >= 0 
            else 'red' for index, row in chart_df.iterrows()]
  fig.add_trace(go.Bar(x=chart_df.index, 
                      y=chart_df['Volume'],
                      marker_color=colors,
                      name='Volume'
                      ), row=2, col=1)

  # Plot MACD trace on 3rd row
  colors = ['green' if val >= 0 
            else '#FF5733' for val in macd.macd_diff()]
  fig.add_trace(go.Bar(x=chart_df.index, 
                      y=macd.macd_diff(),
                      marker_color=colors,
                      name='MACD diff'
                      ), row=3, col=1)
  fig.add_trace(go.Scatter(x=chart_df.index,
                          y=macd.macd(),
                          line=dict(color='orange', width=2),
                          name='MACD'
                          ), row=3, col=1)
  fig.add_trace(go.Scatter(x=chart_df.index,
                          y=macd.macd_signal(),
                          line=dict(color='blue', width=1),
                          name='MACD signal'
                          ), row=3, col=1)


  fig.add_trace(go.Scatter(x=chart_df.index,
                          y=rsi.rsi(),
                          line=dict(color='black', width=2),
                          name='RSI(14)'
                          ), row=4, col=1)

  fig.add_hline(70,line_dash="dot", opacity=0.5, line_width=1, row=4, col=1)
  fig.add_hline(30,line_dash="dot", opacity=0.5, line_width=1, row=4, col=1)


  # Plot stochastics trace on 4th row 
  #fig.add_trace(go.Scatter(x=chart_df.index,
  #                         y=stoch.stoch(),
  #                         line=dict(color='black', width=2)
  #                        ), row=4, col=1)
  #fig.add_trace(go.Scatter(x=chart_df.index,
  #                         y=stoch.stoch_signal(),
  #                         line=dict(color='blue', width=1)
  #                        ), row=4, col=1)


  # removing rangeslider
  fig.update_layout(xaxis_rangeslider_visible=False)

  # removing all empty dates
  # build complete timeline from start date to end date
  dt_all = pd.date_range(start=chart_df.index[0],end=df.index[-1])
  # retrieve the dates that ARE in the original datset
  dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(chart_df.index)]
  # define dates with missing values
  dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

  fig.update_layout(xaxis_rangebreaks=[dict(values=dt_breaks)])
  fig.update_layout(margin=go.layout.Margin(
          l=20, #left margin
          r=20, #right margin
          b=20, #bottom margin
          t=80  #top margin
      ))

  # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
  fig.update_layout(height=900, width=1200, 
                    showlegend=True, 
                    dragmode= 'pan', 
                    xaxis_rangeslider_visible=False,
                    xaxis_rangebreaks=[dict(values=dt_breaks)])


  start_date = "2021-06-01"
  end_date = "2022-01-31"
  zoom_df = chart_df.iloc[chart_df.index >= start_date]
  y_zoom_max = zoom_df["High"].max()
  y_zoom_min = zoom_df["Low"].min()

  fig.update_xaxes(type="date", range=[start_date, end_date])
  fig.update_yaxes(range=[y_zoom_min,y_zoom_max], row=1, col=1)

  # update y-axis label
  fig.update_yaxes(title_text="Price", row=1, col=1)
  fig.update_yaxes(title_text="Volume", row=2, col=1)
  fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
  fig.update_yaxes(title_text="RSI", row=4, col=1)
  fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
  fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
  config = dict({'scrollZoom': True})
  fig.show(config=config) 
  return levels, min_level, max_level, close_price

In [185]:
ticker = yf.Ticker("MU")
df = get_stock_price(ticker,"2019-01-01")
draw_chart(ticker, df, 365)

close_price 81.5 min_level: 80.0 max_level: 82.7
PUT OPTIONS CLOSED: 81.5 SUPPORT -15%: 68.0 RESISTANCE +15%: 95.1
+-----+----------+-------+-------+-----------------+----------+----------------+---------------------+------------------+--------+
|     |   strike |   bid |   ask |   percentChange |   volume |   openInterest |   impliedVolatility | expirationDate   |   mark |
|-----+----------+-------+-------+-----------------+----------+----------------+---------------------+------------------+--------|
| 383 |       75 |  2.41 |  2.44 |         23.9796 |     2142 |          17150 |            0.537114 | 2022-01-21       |  2.425 |
+-----+----------+-------+-------+-----------------+----------+----------------+---------------------+------------------+--------+
CALL OPTIONS CLOSED: 81.5 SUPPORT -15%: 68.0 RESISTANCE +15%: 95.1
+-----+----------+-------+-------+-----------------+----------+----------------+---------------------+------------------+--------+
|     |   strike |   bid |   ask

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning:

divide by zero encountered in double_scalars

